## Programación Lineal
### Plan de Rebalanceo de Carteras

En este cuaderno se muestra la resolución de un problema que plantea determinar que traspasos de dinero
entre fondos de inversión hay que realizar para llevar una cartera actual a una cartera objetivo.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

### Datos
Tenemos disponible un CSV de ejemplo que contiene el valor de las posiciones
de la cartera actual y la objetivo, que nos servirán de ejemplo. Además
tenemos otro CSV con los días que tardaría un traspaso, y que nos servirá
como ejemplo de coste

In [2]:
portfolio = pd.read_csv('../data/portfolio_to_rebalance.csv', index_col='fondo')
portfolio

,current,target
fondo,,
Money Market,16316.22,14824.13
Corporate Bonds,15919.02,17849.97
Europe Equities,7686.84,25799.55
Emerging Markets,26167.30,28407.41
Real Estate,21812.20,11435.97
Commodities,12098.42,1682.97


In [3]:
portfolio.sum()

current    100000.0
target     100000.0
dtype: float64

In [4]:
dias_traspaso = pd.read_csv('../data/coste_traspaso.csv', index_col=0)
dias_traspaso

,Money Market,Corporate Bonds,Europe Equities,Emerging Markets,Real Estate,Commodities
Money Market,2,3,1,3,2,3
Corporate Bonds,1,3,3,2,2,2
Europe Equities,3,1,2,3,2,3
Emerging Markets,3,1,2,2,2,1
Real Estate,3,1,2,2,3,1
Commodities,1,3,2,3,1,1


In [5]:
def rebalancing_series(source, target):
    """
    Calcula los flujos de entrada y salida para rebalancear
    la cartera a partir de la diferencia entre la actual
    y el objetivo
    """
    diff = target - source
    outflow = -diff[diff < 0]
    inflow = diff[diff > 0]
    
    return outflow, inflow

In [6]:
outfunds, infunds = rebalancing_series(portfolio.current, portfolio.target)

In [7]:
outfunds

fondo
Money Market     1492.09
Real Estate     10376.23
Commodities     10415.45
dtype: float64

In [8]:
infunds

fondo
Corporate Bonds      1930.95
Europe Equities     18112.71
Emerging Markets     2240.11
dtype: float64

Seleccionamos de los datos de coste los necesarios para construir
el problema

In [9]:
costes_traspaso = dias_traspaso.loc[outfunds.index, infunds.index]
costes_traspaso

fondo,Corporate Bonds,Europe Equities,Emerging Markets
fondo,,,
Money Market,3,1,3
Real Estate,1,2,2
Commodities,3,2,3


### Implementación del modelo de optimización


En este caso hemos hecho coincidir el valor de la carteras.  Las
restricciones con desigualdad nos permitirían encontrar soluciones
en los casos por ejemplo que tenemos holgura en la cartera actual

In [10]:
out_n = outfunds.shape[0]
in_n = infunds.shape[0]

In [11]:
fundmove = cp.Variable((out_n, in_n))

In [13]:
fundmove

Variable((3, 3))

In [14]:
objetivo = cp.sum(cp.multiply(fundmove, costes_traspaso.values))

In [15]:
constraints = []

for out_i, ofund in enumerate(outfunds):
    o_const = cp.sum(fundmove[out_i, :]) <= ofund
    constraints.append(o_const)
for in_j, infund in enumerate(infunds):
    in_const = cp.sum(fundmove[:, in_j]) >= infund
    constraints.append(in_const)

In [16]:
constraints.append(fundmove >= 0)

In [22]:
constraints.append(fundmove <= 9000)

In [23]:
constraints

[Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Variable((3, 3)))]

In [24]:
problem = cp.Problem(cp.Minimize(objetivo), constraints)

In [25]:
out_prob = problem.solve()

In [26]:
out_prob

42559.950000858495

Reconstruimos la solución a partir de los valores que se quedan 
almacenados en las variables de decisión

In [27]:
resultado = pd.DataFrame(fundmove.value.round(2),
                         index=outfunds.index,
                         columns=infunds.index)    
resultado

fondo,Corporate Bonds,Europe Equities,Emerging Markets
fondo,,,
Money Market,-0.00,1492.09,-0.00
Real Estate,1930.95,7620.62,824.66
Commodities,0.00,9000.00,1415.45


___

### Propuesta Ejercicios
1. Modificar el modelo para que de forma puntual ningún traspaso
sea mayor de 9000€.
